In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal
from keras import regularizers

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

169017344/169001437 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def vgg16(act):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('softmax'))

    return model

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = vgg16('relu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 82s 99ms/step - loss: 7.9613 - accuracy: 0.0106 - top3_acc: 0.0315 - top_k_categorical_accuracy: 0.0530 - val_loss: 6.8319 - val_accuracy: 0.0081 - val_top3_acc: 0.0302 - val_top_k_categorical_accuracy: 0.0518
Epoch 2/50
313/313 [==============================] - 28s 91ms/step - loss: 7.2250 - accuracy: 0.0192 - top3_acc: 0.0542 - top_k_categorical_accuracy: 0.0867 - val_loss: 6.7070 - val_accuracy: 0.0248 - val_top3_acc: 0.0616 - val_top_k_categorical_accuracy: 0.0987
Epoch 3/50
313/313 [==============================] - 28s 90ms/step - loss: 6.9475 - accuracy: 0.0279 - top3_acc: 0.0788 - top_k_categorical_accuracy: 0.1197 - val_loss: 6.6834 - val_accuracy: 0.0177 - val_top3_acc: 0.0593 - val_top_k_categorical_accuracy: 0.1012
Epoch 4/50
313/313 [==============================] - 28s 89ms/step - loss: 6.7696 - accuracy: 0.0394 - top3_acc: 0.1016 - top_k_categorical_accuracy: 0.1559 - val_loss: 6.5695 - val_accuracy: 0.0263 - val_to

In [8]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [7.729734420776367, 7.1569600105285645, 6.906226634979248, 6.735522747039795, 6.600096702575684, 6.511033535003662, 6.407034873962402, 6.332050800323486, 6.257588863372803, 6.1912126541137695, 6.126110553741455, 6.070921897888184, 6.004239559173584, 5.945972442626953, 5.8809614181518555, 5.824333190917969, 5.7656331062316895, 5.69864559173584, 5.644565105438232, 5.584532737731934, 5.520832061767578, 5.453485488891602, 5.402429103851318, 5.337864398956299, 5.291219234466553, 5.223597526550293, 5.16128396987915, 5.1115922927856445, 5.059945583343506, 5.017101287841797, 4.967563629150391, 4.919062614440918, 4.870182514190674, 4.804478645324707, 4.766251087188721, 4.711299896240234, 4.662263870239258, 4.6145758628845215, 4.569904327392578, 4.534721374511719, 4.490486145019531, 4.439613342285156, 4.387755870819092, 4.3472113609313965, 4.316227912902832, 4.264671802520752, 4.219183444976807, 4.188995361328125, 4.146381855010986, 4.108191967010498], 'accuracy': [0.012275000102818012,